In [1]:
!pip install langchain -q --user 

In [2]:
!pip install transformers -q --user 

In [3]:
!pip install pypdf -q --user

In [4]:
!pip install sentence-transformers -q --user

In [5]:
!pip install chromadb -q --user

In [6]:
!pip install google-cloud -q --user

In [12]:
!pip install intel_extension_for_pytorch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 23.3 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
import logging
import os
import random
import re

In [8]:
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings("ignore")

In [10]:
import torch

In [13]:
import intel_extension_for_pytorch as ipex

In [14]:
# Import necessary modules and classes
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
def check_system_config():
    system_type = None
    is_xpu = False
    is_gpu = False
    ret_val = {}
    # random seed
    if torch.cuda.is_available():
        print("GPU is available")
        system_type = "GPU"
        is_gpu = True
        seed = 88
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    else:
        if torch.xpu.is_available():
            print("GPU is not available")
            print("XPU is available")
            system_type = "XPU"
            is_xpu = True
            seed = 88
            random.seed(seed)
            torch.xpu.manual_seed(seed)
            torch.xpu.manual_seed_all(seed)
        else:
             print("XPU is not available")
        

    ret_val = {"is_xpu":is_xpu, "is_gpu": is_gpu, "system_type":system_type}
    print(f"System Config: {ret_val}")
    return ret_val

In [16]:
system_config = check_system_config()
print(system_config)

XPU is not available
System Config: {'is_xpu': False, 'is_gpu': False, 'system_type': None}
{'is_xpu': False, 'is_gpu': False, 'system_type': None}


In [17]:
torch.device("xpu")

device(type='xpu')

In [18]:
# Initialize a directory loader to load PDF documents from a directory# Define the directory where the Chroma database will persist its data
persist_directory = "choma-db"

# Make sure the directory exists, create if it doesn't
import os
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)


In [24]:
# Initialize a directory loader to load PDF documents from a directory
loader = DirectoryLoader("law_data", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Initialize a text splitter to split documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)

# Split the loaded documents into chunks
texts = text_splitter.split_documents(documents)

In [19]:
# texts

In [25]:
# Creating a Vector DB using Chroma DB and SentenceTransformerEmbeddings
# Initialize SentenceTransformerEmbeddings with a pre-trained model
embeddings = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

2023-12-05 18:12:37,853 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: multi-qa-mpnet-base-dot-v1
2023-12-05 18:12:38,583 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu


In [26]:
# Create a Chroma vector database from the text chunks
db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)

2023-12-05 18:12:43,870 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Batches:   0%|          | 0/474 [00:00<?, ?it/s]

In [27]:
# To save and load the saved vector db (if needed in the future)
# Persist the database to disk
db.persist()
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [28]:
# Specify the checkpoint for the language model
checkpoint = "MBZUAI/LaMini-Flan-T5-783M"

# Initialize the tokenizer and base model for text generation
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float32
)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [29]:
# Create a text generation pipeline
pipe = pipeline(
    'text2text-generation',
    model = base_model,
    tokenizer = tokenizer,
    max_length = 512,
    do_sample = True,
    temperature = 0.3,
    top_p= 0.95
)


In [30]:
# RetrievalQA chain

In [31]:
# Initialize a local language model pipeline
local_llm = HuggingFacePipeline(pipeline=pipe)
# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
    return_source_documents=True,
)

In [32]:
# Prompt the user for a query
input_query = str(input("Enter your query:"))

# Execute the query using the QA chain
llm_response = qa_chain({"query": input_query})

# Print the response
print(llm_response['result'])

Enter your query: what is Indian penal code


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Indian penal code is a comprehensive interpretation clause that defines and explains the leading terms used in the chapter, and the meanings announced are consistently adhered to throughout the subsequent chapters.


In [34]:
# Prompt the user for a query
input_query = str(input("Enter your query:"))

# Execute the query using the QA chain
llm_response = qa_chain({"query": input_query})

# Print the response
print(llm_response['result'])

Enter your query: explain ipc 1860


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IPC, 1860 is a civil law code in India that provides a framework for criminal proceedings and punishments. It includes provisions for territorial operation of the code, general explanations, penalties, exceptions, general exceptions, and specific offences. The conviction of an appellant under section 302 of the code cannot be set aside merely for the absence of framing of a specific/alternate charge for an offense punishable under that section.
